In [1]:
import numpy as np
import torch

from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_sample_weight
from numpy import random

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

import os
from pathlib import Path
home = str(Path.home())

In [2]:
def LR(X_train, y_train, X_test, y_test):
    scaler  = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test  = scaler.transform(X_test)
    clf = LogisticRegressionCV(max_iter=100, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [3]:
def RF(X_train, y_train, X_test, y_test):
    clf = RandomForestClassifier(n_estimators=300, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [21]:
factor = 0.8
nr_mean = 3
mean_tables = []
nr_samples=2000

attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
# datasets = ['cifar10']
datasets = ['cifar10', 'cifar100', 'imagenet']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
detectors = ['multilid']
clfs = ['lr', 'rf']

results = {}

for it in range(nr_mean):
    random_state = [21, 30, 65][it] # random.randint(100)
    print("random state: ", random_state)
    final_table = np.zeros((len(attacks), 2))
    base_path = os.path.join(home, 'workspace/multiLID/data/extract/run_{}/'.format(it+1))
    
    results[it] = {}
    
    for dataset in datasets:
        
        if dataset == 'imagenet':
            k=30
        else:
            k=20
        
        for model in models:
            
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
                
            if not dataset in results[it]:
                results[it][dataset] = {}
            if not model in results[it][dataset]:
                results[it][dataset][model] = {}
                

            
            for detector in detectors:
                
                nor_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_normalos_8255.pt"))
                adv_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_adverlos_8255.pt"))
                nor_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_normalos_8255.pt"))
                adv_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_adverlos_8255.pt"))
                nor_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_normalos_8255.pt"))
                adv_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_adverlos_8255.pt"))
                nor_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_normalos_8255.pt"))
                adv_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_adverlos_8255.pt"))
                nor_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_normalos.pt"))
                adv_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_adverlos.pt"))
                nor_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_normalos.pt"))
                adv_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_adverlos.pt"))

                nor = [nor_fgsm, nor_bim, nor_pgd, nor_aa, nor_df, nor_cw]
                adv = [adv_fgsm, adv_bim, adv_pgd, adv_aa, adv_df, adv_cw]
                 
                for rows in range(len(nor)):
                    X_nor = nor[rows]
                    X_adv = adv[rows]
                    if len(X_nor.shape) > 2:
                        X_nor = nor[rows].reshape((nor[rows].shape[0], -1))
                        X_adv = adv[rows].reshape((adv[rows].shape[0], -1))

                    y_nor = np.zeros(X_nor.shape[0]).astype('int')
                    y_adv = np.ones(X_nor.shape[0]).astype('int')

                    x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(X_nor, y_nor, test_size=1-factor, train_size=factor, random_state=random_state)
                    x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(X_adv, y_adv, test_size=1-factor, train_size=factor, random_state=random_state)

                    X_train = np.concatenate((x_train_n, x_train_a))
                    y_train = np.concatenate((y_train_n, y_train_a))

                    X_test = np.concatenate((x_test_n, x_test_a))
                    y_test = np.concatenate((y_test_n, y_test_a))
                    for clf in clfs:
                        if clf == 'lr':
                            y_hat, y_hat_pr = LR(X_train, y_train, X_test, y_test)
                        elif  clf == 'rf':
                            y_hat, y_hat_pr = RF(X_train, y_train, X_test, y_test)

                        auc = round(100*roc_auc_score(y_test, y_hat_pr), 2)
                        f1 =  round(100*f1_score(y_test, y_hat), 2)
                        
                        if not clf in results[it][dataset][model]:
                            results[it][dataset][model][clf] = {}
                        if not attacks[rows] in results[it][dataset][model][clf]:
                            results[it][dataset][model][clf][attacks[rows]] = {}
                        if not attacks[rows] in results[it][dataset][model][clf]:
                            results[it][dataset][model][clf][attacks[rows]] = {}

                        # if not 'auc' in results[it][dataset][model][clf][attacks[rows]]:
                        #     results[it][dataset][model][clf][attacks[rows]]['auc'] = {}
                        # if not 'f1' in results[it][dataset][model][clf][attacks[rows]]:
                        #     results[it][dataset][model][clf][attacks[rows]]['f1' ] = {}

                        results[it][dataset][model][clf][attacks[rows]]['auc'] = auc
                        results[it][dataset][model][clf][attacks[rows]]['f1'] = f1                    

                    # print(attacks[rows].upper(), '&' , auc, '&', f1)

random state:  21


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

lr fgsm
rf fgsm


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

lr bim
rf bim


...

lr pgd
rf pgd
lr aa
rf aa


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

lr df
rf df


/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/lorenzp/.conda/envs/cuda--11-1-1--pytorch--1-9-0/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

lr cw
rf cw


# LR

In [27]:
lid_list_mean = []
for it_mean in range(nr_mean):
    lid_list = []
    for dataset in ['cifar10', 'cifar100', 'imagenet']:
        for model in ['wrn28-10', 'vgg16', 'wrn50-2']:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            tmp = np.zeros((len(attacks), 2))
            for it, val in enumerate(results[it_mean][dataset][model]['lr'].items()):
                # print(it, val)
                tmp[it, 0] = val[1]['auc'].copy()
                tmp[it, 1] = val[1]['f1'].copy()
            lid_list.append(tmp.copy())
    
    lid_list_mean.append(np.stack(lid_list))
lid_list_mean_res = np.stack(lid_list_mean)
lid_mean = np.mean(lid_list_mean_res, axis=0)
lid_var = np.var(lid_list_mean_res, axis=0)

In [28]:
lid_mean.shape[0]

5

In [29]:
latex_table = ""
for j in range(lid_mean.shape[1]):
    latex_table += "\\textbf{"+f"{attacks[j].upper()}"+"} & "
    for i in range(lid_mean.shape[0]):
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
    latex_table = latex_table[:-2] + "\\\\ \n"

print(latex_table)

\textbf{FGSM} & $97.94 \pm 0.09$ & $92.31 \pm 0.81$ & $93.21 \pm 0.06$ & $85.64 \pm 0.25$ & $99.23 \pm 0.02$ & $96.16 \pm 0.29$ & $85.44 \pm 0.00$ & $78.63 \pm 0.46$ & $80.83 \pm 2.23$ & $73.32 \pm 1.93$ \\ 
\textbf{BIM} & $95.74 \pm 0.19$ & $90.97 \pm 0.04$ & $95.03 \pm 0.66$ & $89.05 \pm 1.44$ & $98.45 \pm 0.04$ & $93.49 \pm 0.12$ & $84.47 \pm 0.62$ & $77.25 \pm 1.68$ & $97.05 \pm 0.00$ & $91.12 \pm 0.28$ \\ 
\textbf{PGD} & $97.07 \pm 0.39$ & $92.19 \pm 0.13$ & $93.09 \pm 0.68$ & $86.27 \pm 1.39$ & $99.09 \pm 0.00$ & $96.04 \pm 0.11$ & $89.05 \pm 0.21$ & $81.65 \pm 0.67$ & $97.55 \pm 0.04$ & $92.69 \pm 0.06$ \\ 
\textbf{AA} & $99.26 \pm 0.10$ & $96.94 \pm 0.68$ & $98.56 \pm 0.12$ & $94.14 \pm 1.03$ & $99.93 \pm 0.00$ & $98.80 \pm 0.05$ & $91.31 \pm 0.54$ & $83.35 \pm 0.66$ & $99.95 \pm 0.00$ & $99.38 \pm 0.01$ \\ 
\textbf{DF} & $98.00 \pm 0.04$ & $94.20 \pm 0.03$ & $89.48 \pm 0.84$ & $83.60 \pm 1.47$ & $75.83 \pm 0.98$ & $70.94 \pm 2.66$ & $70.54 \pm 4.46$ & $65.71 \pm 3.02$ & $55.01

# multiLID

In [25]:
lid_list_mean = []
for it_mean in range(nr_mean):
    lid_list = []
    for dataset in ['cifar10', 'cifar100', 'imagenet']:
        for model in ['wrn28-10', 'vgg16', 'wrn50-2']:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            tmp = np.zeros((len(attacks), 2))
            for it, val in enumerate(results[it_mean][dataset][model]['rf'].items()):
                # print(it, val)
                tmp[it, 0] = val[1]['auc'].copy()
                tmp[it, 1] = val[1]['f1'].copy()
            lid_list.append(tmp.copy())
    
    lid_list_mean.append(np.stack(lid_list))
lid_list_mean_res = np.stack(lid_list_mean)
lid_mean = np.mean(lid_list_mean_res, axis=0)
lid_var = np.var(lid_list_mean_res, axis=0)

In [26]:
latex_table = ""
for j in range(lid_mean.shape[1]):
    latex_table += "\\textbf{"+f"{attacks[j].upper()}"+"} & "
    for i in range(lid_mean.shape[0]):
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
    latex_table = latex_table[:-2] + "\\\\\n"

# Print the LaTeX table
print(latex_table)

\textbf{FGSM} & $97.05 \pm 0.17$ & $91.43 \pm 0.88$ & $91.25 \pm 0.04$ & $83.53 \pm 0.44$ & $98.67 \pm 0.23$ & $95.39 \pm 0.58$ & $83.18 \pm 0.83$ & $77.55 \pm 0.34$ & $80.67 \pm 1.59$ & $74.88 \pm 0.56$ \\
\textbf{BIM} & $96.43 \pm 0.20$ & $90.20 \pm 0.48$ & $94.73 \pm 0.30$ & $88.00 \pm 0.80$ & $97.78 \pm 0.06$ & $91.83 \pm 0.02$ & $83.61 \pm 0.14$ & $76.13 \pm 2.07$ & $94.76 \pm 0.24$ & $87.95 \pm 0.27$ \\
\textbf{PGD} & $97.80 \pm 0.10$ & $93.26 \pm 0.03$ & $92.42 \pm 1.72$ & $84.74 \pm 3.94$ & $98.80 \pm 0.06$ & $94.69 \pm 1.29$ & $88.72 \pm 0.27$ & $82.36 \pm 1.25$ & $96.88 \pm 0.08$ & $91.06 \pm 0.01$ \\
\textbf{AA} & $99.62 \pm 0.00$ & $97.60 \pm 0.02$ & $98.74 \pm 0.03$ & $95.08 \pm 1.12$ & $99.88 \pm 0.00$ & $98.45 \pm 0.00$ & $91.76 \pm 0.33$ & $85.16 \pm 0.01$ & $99.87 \pm 0.01$ & $99.00 \pm 0.03$ \\
\textbf{DF} & $97.69 \pm 0.15$ & $94.48 \pm 0.17$ & $89.41 \pm 1.61$ & $84.29 \pm 1.00$ & $76.51 \pm 0.26$ & $72.02 \pm 0.24$ & $75.57 \pm 0.57$ & $70.74 \pm 0.71$ & $53.61 \pm